# VacationPy
----


In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
import csv

city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,lat,lng,Humidity,Wind Speed,Max Temp,Cloudiness
0,0,Hermanus,ZA,1585697097,-34.42,19.23,89,11.95,61.0,{'all': 34}
1,1,Ushuaia,AR,1585696974,-54.80,-68.30,76,12.75,48.2,{'all': 40}
2,2,Vestmanna,FO,1585697097,62.16,-7.17,87,28.86,44.6,{'all': 100}
3,3,Husavik,IS,1585697448,66.04,-17.34,51,39.15,39.2,{'all': 90}
4,4,Tasiilaq,GL,1585697448,65.61,-37.64,29,29.97,24.8,{'all': 2}


In [22]:
print(len(city_data_df))

2473


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)

In [16]:
locations = city_data_df[["lat", "lng"]]

humidity = city_data_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal Conditions:
    
Max temp 80 degrees but higher than 70
Humidity less than 50
Wind speed less than 20 mph.

In [18]:



ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 20) \
                                    & (city_data_df["Humidity"]< 50)].dropna()
print(len(ideal_weather))

18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = ideal_weather[["City", "Country", "lat", "lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,lat,lng,Hotel Name
5,Selma,US,36.57,-119.61,
54,Jāwad,IN,24.60,74.85,
89,Praia,CV,14.92,-23.51,
94,Araouane,ML,18.90,-3.53,
151,Kutum,SD,14.20,24.67,
234,Flagstaff,US,35.20,-111.65,
241,Buin,CL,-33.73,-70.75,
366,Taoudenni,ML,22.68,-3.98,
389,Pisco,PE,-13.70,-76.22,
400,Adrar,MR,20.50,-10.07,


In [30]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,lat,lng,Hotel Name
5,Selma,US,36.57,-119.61,Holiday Inn Selma-Swancourt
54,Jāwad,IN,24.60,74.85,हर मंदिर जावद
89,Praia,CV,14.92,-23.51,LT Aparthotel
94,Araouane,ML,18.90,-3.53,
151,Kutum,SD,14.20,24.67,Dibbo areaمنطقة دبو
234,Flagstaff,US,35.20,-111.65,Drury Inn & Suites Flagstaff
241,Buin,CL,-33.73,-70.75,Paihuen
366,Taoudenni,ML,22.68,-3.98,
389,Pisco,PE,-13.70,-76.22,Hostal San Isidro Oficial
400,Adrar,MR,20.50,-10.07,


In [31]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [32]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))